<a href="https://colab.research.google.com/github/ruany-doehnert/Deforestation_Amazon/blob/main/data_source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install and authenticate Earth Engine in Colab
# !pip install earthengine-api --quiet
# !pip install rasterio
import rasterio
import numpy as np
import pandas as pd
import ee

In [ ]:
# conect with google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ee.Authenticate()
ee.Initialize(project='amazon-deforestation-462101')

# Define region of interest (example: small Amazon region)
region = ee.Geometry.BBox(-60, -10, -59, -9)

# Hansen treecover
gfc = ee.Image('UMD/hansen/global_forest_change_2022_v1_10')
treecover = gfc.select('treecover2000')

# Landsat 8 median composite for 2021
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2021-01-01', '2021-12-31') \
    .filterBounds(region) \
    .median()

# Calculate NDVI and NBR
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
nbr = landsat.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR')

# Elevation
elevation = ee.Image('USGS/SRTMGL1_003').rename('elevation')

# Stack features
features = treecover.rename('treecover') \
    .addBands([ndvi, nbr, elevation])

# Sample points in the region
samples = features.sample(
    region=region,
    scale=30,
    numPixels=2000,
    seed=42,
    geometries=True
)

# Create label: forest if treecover > 30
def label_fn(f):
    return f.set('label', ee.Number(f.get('treecover')).gt(30).int())
samples = samples.map(label_fn)

# Export to Drive
task = ee.batch.Export.table.toDrive(
    collection=samples,
    description='export_training_dataset_with_features',
    folder='earthengine_export',
    fileNamePrefix='training_data_amazon_features',
    fileFormat='CSV',
    selectors=['treecover', 'NDVI', 'NBR', 'elevation', 'label']
)
task.start()
print("🚀 Export started! Check your Google Drive shortly.")

🚀 Export started! Check your Google Drive shortly.
